In [1]:
import os
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from data_loader import DataLoader

In [3]:
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dropout, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


# Generator

In [4]:
def conv2d(layer_input, filters, f_size=4, normalize=True):
    """Layers used during downsampling"""
    d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    if normalize:
        d = InstanceNormalization()(d)
    return d

def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
    """Layers used during upsampling"""
    u = UpSampling2D(size=2)(layer_input)
    u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
    if dropout_rate:
        u = Dropout(dropout_rate)(u)
    u = InstanceNormalization()(u)
    u = Concatenate()([u, skip_input])
    return u

In [5]:
def build_generator(img_shape, channels, gf):
    """U-Net Generator"""

    # Image input
    d0 = Input(shape=img_shape)

    # Downsampling
    d1 = conv2d(d0, gf, normalize=False)
    d2 = conv2d(d1, gf*2)
    d3 = conv2d(d2, gf*4)
    d4 = conv2d(d3, gf*8)
    d5 = conv2d(d4, gf*8)
    d6 = conv2d(d5, gf*8)
    d7 = conv2d(d6, gf*8)

    # Upsampling
    u1 = deconv2d(d7, d6, gf*8)
    u2 = deconv2d(u1, d5, gf*8)
    u3 = deconv2d(u2, d4, gf*8)
    u4 = deconv2d(u3, d3, gf*4)
    u5 = deconv2d(u4, d2, gf*2)
    u6 = deconv2d(u5, d1, gf)

    u7 = UpSampling2D(size=2)(u6)
    output_img = Conv2D(channels, kernel_size=4, strides=1,
                        padding='same', activation='tanh')(u7)

    return Model(d0, output_img)

# Discriminator

In [6]:
def d_layer(layer_input, filters, f_size=4, normalization=True):
    """Discriminator layer"""
    d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
    d = LeakyReLU(alpha=0.2)(d)
    if normalization:
        d = InstanceNormalization()(d)
    return d

In [7]:
def build_discriminator(img_shape, df):
    img = Input(shape=img_shape)

    d1 = d_layer(img, df, normalization=False)
    d2 = d_layer(d1, df*2)
    d3 = d_layer(d2, df*4)
    d4 = d_layer(d3, df*8)

    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

    return Model(img, validity)

# Helper function

In [8]:
def sample_images(data_loader, dataset_name, g_AB, g_BA, epoch, batch_i):
    os.makedirs('images/%s' % dataset_name, exist_ok=True)
    r, c = 2, 3

    imgs_A, imgs_B = data_loader.load_data(batch_size=1, is_testing=True)

    # Translate images to the other domain
    fake_B = g_AB.predict(imgs_A)
    fake_A = g_BA.predict(imgs_B)
    # Translate back to original domain
    reconstr_A = g_BA.predict(fake_B)
    reconstr_B = g_AB.predict(fake_A)

    gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    titles = ['Original', 'Translated', 'Reconstructed']
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i, j].set_title(titles[j])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%s/%d_%d.png" % (dataset_name, epoch, batch_i))
    plt.close()

# Train

In [9]:
def train(data_loader, dataset_name, 
          g_AB, g_BA, d_A, d_B, combined, 
          disc_patch,
          epochs, batch_size=128, sample_interval=50):

    start_time = datetime.datetime.now()

    # Adversarial loss ground truths
    valid = np.ones((batch_size,) + disc_patch)
    fake = np.zeros((batch_size,) + disc_patch)

    for epoch in range(epochs):
        for batch_i, (imgs_A, imgs_B) in enumerate(data_loader.load_batch(batch_size)):
            print("epoch", epoch, "- batch #", batch_i)

            # ----------------------
            #  Train Discriminators
            # ----------------------

            # Translate images to opposite domain
            fake_B = g_AB.predict(imgs_A)
            fake_A = g_BA.predict(imgs_B)

            # Train the discriminators (original images = real / translated = Fake)
            dA_loss_real = d_A.train_on_batch(imgs_A, valid)
            dA_loss_fake = d_A.train_on_batch(fake_A, fake)
            dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

            dB_loss_real = d_B.train_on_batch(imgs_B, valid)
            dB_loss_fake = d_B.train_on_batch(fake_B, fake)
            dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

            # Total disciminator loss
            d_loss = 0.5 * np.add(dA_loss, dB_loss)

            # ------------------
            #  Train Generators
            # ------------------

            # Train the generators
            g_loss = combined.train_on_batch([imgs_A, imgs_B], [valid, valid, \
                                                                     imgs_B, imgs_A, \
                                                                     imgs_A, imgs_B])

            elapsed_time = datetime.datetime.now() - start_time
            # Plot the progress
            print ("[%d] [%d/%d] time: %s, [d_loss: %f, g_loss: %f]" % (epoch, batch_i,
                                                                        data_loader.n_batches,
                                                                        elapsed_time,
                                                                        d_loss[0], g_loss[0]))

            # If at save interval => save generated image samples
            if batch_i % sample_interval == 0:
                sample_images(data_loader, dataset_name, g_AB, g_BA, epoch, batch_i)

# main()

In [10]:
# Input shape
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)

In [11]:
# Configure data loader
dataset_name = 'edges2shoes'
data_loader = DataLoader(dataset_name=dataset_name,
                         img_res=(img_rows, img_cols))

In [12]:
# Calculate output shape of D (PatchGAN)
patch = int(img_rows / 2**4)
disc_patch = (patch, patch, 1)

In [13]:
# Number of filters in the first layer of G and D
gf = 64
df = 64

In [14]:
# create optimizer
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [15]:
# Build and compile the discriminators
d_A = build_discriminator(img_shape, df)
d_B = build_discriminator(img_shape, df)
d_A.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
d_B.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])

In [16]:
#-------------------------
# Construct Computational
#   Graph of Generators
#-------------------------

# Build the generators
g_AB = build_generator(img_shape, channels, gf)
g_BA = build_generator(img_shape, channels, gf)

In [17]:
# Input images from both domains
img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)

In [18]:
# Translate images to the other domain
fake_B = g_AB(img_A)
fake_A = g_BA(img_B)

In [19]:
# Translate images back to original domain
reconstr_A = g_BA(fake_B)
reconstr_B = g_AB(fake_A)

In [20]:
# For the combined model we will only train the generators
d_A.trainable = False
d_B.trainable = False

In [21]:
# Discriminators determines validity of translated images
valid_A = d_A(fake_A)
valid_B = d_B(fake_B)

In [22]:
# Objectives
# + Adversarial: Fool domain discriminators
# + Translation: Minimize MAE between e.g. fake B and true B
# + Cycle-consistency: Minimize MAE between reconstructed images and original
combined = Model(inputs=[img_A, img_B],
                      outputs=[ valid_A, valid_B,
                                fake_B, fake_A,
                                reconstr_A, reconstr_B ])
combined.compile(loss=['mse', 'mse',
                       'mae', 'mae',
                       'mae', 'mae'],
                 optimizer=optimizer)

In [ ]:
# epochs=20
epochs=5
train(data_loader, dataset_name, 
      g_AB, g_BA, d_A, d_B, combined,
      disc_patch,
      epochs=epochs, batch_size=1, sample_interval=200)

epoch 0 # 0


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] [0/49825] time: 0:00:02.753157, [d_loss: 0.325678, g_loss: 3.150854]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


epoch 0 # 1
[0] [1/49825] time: 0:00:06.512421, [d_loss: 0.272776, g_loss: 2.676764]
epoch 0 # 2
[0] [2/49825] time: 0:00:09.725880, [d_loss: 0.245721, g_loss: 2.466418]
epoch 0 # 3
[0] [3/49825] time: 0:00:12.909697, [d_loss: 0.258993, g_loss: 1.671192]
epoch 0 # 4
[0] [4/49825] time: 0:00:15.965880, [d_loss: 0.188796, g_loss: 2.790794]
epoch 0 # 5
[0] [5/49825] time: 0:00:19.117180, [d_loss: 0.191661, g_loss: 1.591216]
epoch 0 # 6
[0] [6/49825] time: 0:00:22.311919, [d_loss: 0.196063, g_loss: 2.830503]
epoch 0 # 7
[0] [7/49825] time: 0:00:25.948970, [d_loss: 0.163234, g_loss: 2.287107]
epoch 0 # 8
[0] [8/49825] time: 0:00:29.338723, [d_loss: 0.349116, g_loss: 1.565508]
epoch 0 # 9
[0] [9/49825] time: 0:00:32.507564, [d_loss: 0.321941, g_loss: 1.596231]
epoch 0 # 10
[0] [10/49825] time: 0:00:35.999136, [d_loss: 0.193477, g_loss: 2.406057]
epoch 0 # 11
[0] [11/49825] time: 0:00:39.392599, [d_loss: 0.350549, g_loss: 1.200775]
epoch 0 # 12
[0] [12/49825] time: 0:00:42.521781, [d_loss: 0.

...

[0] [293/49825] time: 0:17:13.726351, [d_loss: 0.223326, g_loss: 1.932297]
epoch 0 # 294
[0] [294/49825] time: 0:17:18.116083, [d_loss: 0.251957, g_loss: 2.124737]
epoch 0 # 295
[0] [295/49825] time: 0:17:22.444956, [d_loss: 0.274821, g_loss: 1.943497]
epoch 0 # 296
[0] [296/49825] time: 0:17:26.761909, [d_loss: 0.378798, g_loss: 1.657369]
epoch 0 # 297
[0] [297/49825] time: 0:17:31.203460, [d_loss: 0.220628, g_loss: 1.743078]
epoch 0 # 298
[0] [298/49825] time: 0:17:35.604905, [d_loss: 0.198750, g_loss: 1.785110]
epoch 0 # 299
[0] [299/49825] time: 0:17:39.880102, [d_loss: 0.170183, g_loss: 1.985327]
epoch 0 # 300
[0] [300/49825] time: 0:17:44.209004, [d_loss: 0.260961, g_loss: 1.567906]
epoch 0 # 301
[0] [301/49825] time: 0:17:48.377983, [d_loss: 0.245759, g_loss: 1.519315]
epoch 0 # 302
[0] [302/49825] time: 0:17:52.541668, [d_loss: 0.183408, g_loss: 1.661764]
epoch 0 # 303
[0] [303/49825] time: 0:17:56.809263, [d_loss: 0.225610, g_loss: 1.806934]
epoch 0 # 304
[0] [304/49825] time:

[0] [386/49825] time: 0:24:05.534431, [d_loss: 0.284081, g_loss: 1.763181]
epoch 0 # 387
[0] [387/49825] time: 0:24:09.919843, [d_loss: 0.263585, g_loss: 1.776706]
epoch 0 # 388
[0] [388/49825] time: 0:24:14.544339, [d_loss: 0.283995, g_loss: 1.594035]
epoch 0 # 389
[0] [389/49825] time: 0:24:18.636184, [d_loss: 0.273503, g_loss: 1.506721]
epoch 0 # 390
[0] [390/49825] time: 0:24:22.723458, [d_loss: 0.271574, g_loss: 1.426019]
epoch 0 # 391
[0] [391/49825] time: 0:24:26.695458, [d_loss: 0.281800, g_loss: 1.279131]
epoch 0 # 392
[0] [392/49825] time: 0:24:30.758965, [d_loss: 0.297837, g_loss: 1.696634]
epoch 0 # 393
[0] [393/49825] time: 0:24:34.792819, [d_loss: 0.281390, g_loss: 1.417174]
epoch 0 # 394
[0] [394/49825] time: 0:24:38.788473, [d_loss: 0.265855, g_loss: 2.348038]
epoch 0 # 395
[0] [395/49825] time: 0:24:42.767907, [d_loss: 0.256570, g_loss: 1.820993]
epoch 0 # 396
[0] [396/49825] time: 0:24:46.672649, [d_loss: 0.262417, g_loss: 1.779912]
epoch 0 # 397
[0] [397/49825] time:

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


epoch 0 # 401
[0] [401/49825] time: 0:25:06.911313, [d_loss: 0.255393, g_loss: 1.447951]
epoch 0 # 402
[0] [402/49825] time: 0:25:11.240728, [d_loss: 0.314675, g_loss: 1.302233]
epoch 0 # 403
[0] [403/49825] time: 0:25:15.704398, [d_loss: 0.268316, g_loss: 1.206818]
epoch 0 # 404
[0] [404/49825] time: 0:25:20.017077, [d_loss: 0.273457, g_loss: 1.115051]
epoch 0 # 405
[0] [405/49825] time: 0:25:24.485058, [d_loss: 0.274258, g_loss: 1.457419]
epoch 0 # 406
[0] [406/49825] time: 0:25:28.485005, [d_loss: 0.266887, g_loss: 1.367303]
epoch 0 # 407
[0] [407/49825] time: 0:25:32.482032, [d_loss: 0.260905, g_loss: 1.380377]
epoch 0 # 408
[0] [408/49825] time: 0:25:36.405996, [d_loss: 0.278410, g_loss: 1.342741]
epoch 0 # 409
[0] [409/49825] time: 0:25:40.310018, [d_loss: 0.244474, g_loss: 1.688826]
epoch 0 # 410
[0] [410/49825] time: 0:25:44.105181, [d_loss: 0.266758, g_loss: 1.288459]
epoch 0 # 411
[0] [411/49825] time: 0:25:47.904588, [d_loss: 0.235204, g_loss: 1.700740]
epoch 0 # 412
[0] [41

[0] [493/49825] time: 0:31:21.195267, [d_loss: 0.288886, g_loss: 1.486184]
epoch 0 # 494
[0] [494/49825] time: 0:31:25.208686, [d_loss: 0.213629, g_loss: 2.087547]
epoch 0 # 495
[0] [495/49825] time: 0:31:29.393238, [d_loss: 0.200640, g_loss: 1.932274]
epoch 0 # 496
[0] [496/49825] time: 0:31:33.702116, [d_loss: 0.238421, g_loss: 1.573895]
epoch 0 # 497
[0] [497/49825] time: 0:31:37.851781, [d_loss: 0.246118, g_loss: 1.561542]
epoch 0 # 498
[0] [498/49825] time: 0:31:41.848870, [d_loss: 0.220055, g_loss: 1.587565]
epoch 0 # 499
[0] [499/49825] time: 0:31:45.858842, [d_loss: 0.197991, g_loss: 1.626035]
epoch 0 # 500
[0] [500/49825] time: 0:31:49.873030, [d_loss: 0.269491, g_loss: 1.282886]
epoch 0 # 501
[0] [501/49825] time: 0:31:53.820493, [d_loss: 0.235043, g_loss: 1.385800]
epoch 0 # 502
[0] [502/49825] time: 0:31:57.689806, [d_loss: 0.178551, g_loss: 1.808635]
epoch 0 # 503
[0] [503/49825] time: 0:32:01.532755, [d_loss: 0.205225, g_loss: 1.853810]
epoch 0 # 504
[0] [504/49825] time: